In [1]:
import pandas as pd

In [10]:
import plotly.express as px

In [2]:
df = pd.read_csv('berlin_bikedata_2017-2019.csv', parse_dates=True)

In [3]:
df.head(5)

,timestamp,station,total_bikes,hour,hour_str,weekday,day_name,month,month_name,year,description,lat,lon
0,2017-01-01 00:00:00,02-MI-JAN-N,6,0,0 Uhr,6,Sunday,1,January,2017,Jannowitzbrücke Nord,52.513932,13.417835
1,2017-01-01 00:00:00,02-MI-JAN-S,5,0,0 Uhr,6,Sunday,1,January,2017,Jannowitzbrücke Süd,52.513943,13.417611
2,2017-01-01 00:00:00,03-MI-SAN-O,4,0,0 Uhr,6,Sunday,1,January,2017,Invalidenstraße Ost,52.527177,13.372016
3,2017-01-01 00:00:00,03-MI-SAN-W,5,0,0 Uhr,6,Sunday,1,January,2017,Invalidenstraße West,52.527686,13.373105
4,2017-01-01 00:00:00,05-FK-OBB-O,16,0,0 Uhr,6,Sunday,1,January,2017,Oberbaumbrücke Ost,52.501199,13.445060


In [4]:
df['station_short'] = df.station.str.split("-", n=1, expand=True).rename(columns={0:"station_short"})['station_short']

In [5]:
df.head(5)

,timestamp,station,total_bikes,hour,hour_str,weekday,day_name,month,month_name,year,description,lat,lon,station_short
0,2017-01-01 00:00:00,02-MI-JAN-N,6,0,0 Uhr,6,Sunday,1,January,2017,Jannowitzbrücke Nord,52.513932,13.417835,02
1,2017-01-01 00:00:00,02-MI-JAN-S,5,0,0 Uhr,6,Sunday,1,January,2017,Jannowitzbrücke Süd,52.513943,13.417611,02
2,2017-01-01 00:00:00,03-MI-SAN-O,4,0,0 Uhr,6,Sunday,1,January,2017,Invalidenstraße Ost,52.527177,13.372016,03
3,2017-01-01 00:00:00,03-MI-SAN-W,5,0,0 Uhr,6,Sunday,1,January,2017,Invalidenstraße West,52.527686,13.373105,03
4,2017-01-01 00:00:00,05-FK-OBB-O,16,0,0 Uhr,6,Sunday,1,January,2017,Oberbaumbrücke Ost,52.501199,13.445060,05


In [7]:
df['timestamp'] = pd.to_datetime(df.timestamp)

In [63]:
class Frequency:
    def __init__(self, frequency, frequency_dict, location_id):
        self.frequency = frequency
        self.frequency_short = frequency_dict["frequency"][frequency]["short"]
        self.d3_format = frequency_dict["frequency"][frequency]["d3_format"]
        self.location_id = location_id
        self.hovertext = f'{self.frequency}: %{{x|{self.d3_format}}}<br>Total Bikes: %{{y}}'
        self.title = 

In [77]:
frequency_dict = {"frequency": {"Day": {"short": "D", "d3_format": "%b %d, %Y (%a)"}, "Week": {"short": "W", "d3_format": "%b %d, %Y"}, "Month":{"short": "M", "d3_format": "%B %Y"}, "Year":{"short": "Y", "d3_format": "%Y"}}}

In [85]:
barchart_object = Frequency("Week", frequency_dict, "27")

In [86]:
barchart_df = df.set_index("timestamp").groupby(["description", "station_short"])[["total_bikes"]].resample(barchart_object.frequency_short).sum().reset_index()
barchart_df

,description,station_short,timestamp,total_bikes
0,Alberichstraße,24,2017-01-01,103
1,Alberichstraße,24,2017-01-08,940
2,Alberichstraße,24,2017-01-15,501
3,Alberichstraße,24,2017-01-22,712
4,Alberichstraße,24,2017-01-29,1484
...,...,...,...,...
4103,Yorkstraße West,18,2019-12-08,14418
4104,Yorkstraße West,18,2019-12-15,14375
4105,Yorkstraße West,18,2019-12-22,15009
4106,Yorkstraße West,18,2019-12-29,5298


In [93]:
street_names = " / ".join(barchart_df[barchart_df.station_short == barchart_object.location_id]["description"].unique())
barchart_title= f"{barchart_object.frequency}ly Data for Bicycle Counter {street_names}"
barchart_title

'Weekly Data for bicycle counter Markstraße'

In [95]:
barchart_df.description.unique()

array(['Alberichstraße', 'Berliner Straße Nord', 'Berliner Straße Süd',
       'Breitenbachplatz Ost', 'Breitenbachplatz West',
       'Frankfurter Allee Ost', 'Frankfurter Allee West',
       'Invalidenstraße Ost', 'Invalidenstraße West',
       'Jannowitzbrücke Nord', 'Jannowitzbrücke Süd', 'Kaisersteg',
       'Klosterstraße Nord', 'Klosterstraße Süd',
       'Mariendorfer Damm Nord', 'Mariendorfer Damm Süd', 'Markstraße',
       'Maybachufer', 'Monumentenstraße', 'Oberbaumbrücke Ost',
       'Oberbaumbrücke West', 'Paul-und-Paula-Uferweg',
       'Prinzregentenstraße', 'Schwedter Steg', 'Yorckstraße Ost',
       'Yorkstraße West'], dtype=object)

In [101]:
streets_dict = {"Alberichstraße": "24", "Berliner Straße": "10", "Breitenbachplatz": "17", "Frankfurter Allee": "06", "Invalidenstraße": "03","Jannowitzbrücke": "02", "Kaisersteg": "23", "Klosterstraße": "15", "Mariendorfer Damm": "20", "Markstraße": "27", "Maybachufer": "21", "Monumentenstraße": "19", "Oberbaumbrücke": "05", "Paul-und-Paula-Uferweg": "26", "Prinzregentenstraße": "13", "Schwedter Steg": "12", "Yorckstraße": "18"}

In [102]:
streets_dict.keys()

dict_keys(['Alberichstraße', 'Berliner Straße', 'Breitenbachplatz', 'Frankfurter Allee', 'Invalidenstraße', 'Jannowitzbrücke', 'Kaisersteg', 'Klosterstraße', 'Mariendorfer Damm', 'Markstraße', 'Maybachufer', 'Monumentenstraße', 'Oberbaumbrücke', 'Paul-und-Paula-Uferweg', 'Prinzregentenstraße', 'Schwedter Steg', 'Yorckstraße'])

In [94]:
fig = px.bar(barchart_df[barchart_df.station_short == barchart_object.location_id], x="timestamp", y="total_bikes", color="description", title=barchart_title)
fig.update_traces(hovertemplate=barchart_object.hovertext)
fig.show()